# 11장 사전 학습 모델 활용하기

* "부록3 매트플롯립 입문"에서 한글 폰트를 올바르게 출력하기 위한 설치 방법을 설명했다. 설치 방법은 다음과 같다.

In [ ]:
!sudo apt-get install -y fonts-nanum* | tail -n 1
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

In [ ]:
# 필요 라이브러리 설치

!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

* 모든 설치가 끝나면 한글 폰트를 바르게 출력하기 위해 **[런타임]** -> **[런타임 다시시작]**을 클릭한 다음, 아래 셀부터 코드를 실행해 주십시오.

In [ ]:
# 라이브러리 임포트

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# 폰트 관련 용도
import matplotlib.font_manager as fm

# 나눔 고딕 폰트의 경로 명시
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=10).get_name()

In [ ]:
# 파이토치 관련 라이브러리

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [ ]:
# warning 표시 끄기
import warnings
warnings.simplefilter('ignore')

# 기본 폰트 설정
plt.rcParams['font.family'] = font_name

# 기본 폰트 사이즈 변경
plt.rcParams['font.size'] = 14

# 기본 그래프 사이즈 변경
plt.rcParams['figure.figsize'] = (6,6)

# 기본 그리드 표시
# 필요에 따라 설정할 때는, plt.grid()
plt.rcParams['axes.grid'] = True

# 마이너스 기호 정상 출력
plt.rcParams['axes.unicode_minus'] = False

# 넘파이 부동소수점 자릿수 표시
np.set_printoptions(suppress=True, precision=4)

In [ ]:
# GPU 디바이스 할당

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

### 공통 함수 불러오기

In [ ]:
# 공통 함수 다운로드
!git clone https://github.com/wikibook/pythonlibs.git

# 공통 함수 불러오기
from pythonlibs.torch_lib1 import *

# 공통 함수 확인
print(README)

## 11.4 적응형 풀링 함수(nn.AdaptiveAvgPool2d 함수)

In [ ]:
# nn.AdaptiveAvgPool2d 정의
p = nn.AdaptiveAvgPool2d((1,1)) # 출력을 (1, 1) 크기로 만드는 풀링 레이어 정의
print(p)

# 선형 함수의 정의
l1 = nn.Linear(32, 10) # 입력 32, 출력 10의 선형 레이어 정의
print(l1)

In [ ]:
# 코드 11-3: 사전 학습 모델 시뮬레이션
# 더미 데이터 생성 (배치크기=100, 채널=32, 높이=16, 너비=16)
inputs = torch.randn(100, 32, 16, 16)

m1 = p(inputs)                              # (100, 32, 16, 16) -> (100, 32, 1, 1)
print(m1.shape[0])
m2 = m1.view(m1.shape[0], -1)               # 선형 레이어 입력을 위해 1차원으로 펼침 (Flatten) -> (100, 32)
m3 = l1(m2)                                 # 최종 예측 결과 -> (100, 10)

# shape 확인
print(m1.shape)
print(m2.shape)
print(m3.shape)

## 11.5 데이터 준비

In [ ]:
# 분류 클래스명 정의

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 분류 클래스 수는 10
n_output = len(classes)
n_output

In [ ]:
# Transforms 정의
# 학습 데이터용: 리사이즈, 좌우반전, 텐서 변환, 정규화, 랜덤 지우기 적용
transform_train = transforms.Compose([
    transforms.Resize(112),                 # 이미지 크기를 112x112로 조정
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # 3채널 이미지에 대한 정규화
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

# 검증 데이터용: 리사이즈, 텐서 변환, 정규화만 적용
transform = transforms.Compose([
    transforms.Resize(112),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 3채널 이미지에 대한 정규화
])

In [ ]:
# 데이터 취득용 함수 dataset

data_root = './data'

train_set = datasets.CIFAR10(
    root = data_root, train = True,
    download = True, transform = transform_train)

# 검증 데이터셋
test_set = datasets.CIFAR10(
    root = data_root, train = False,
    download = True, transform = transform)

In [ ]:
# 배치 사이즈 지정
batch_size = 50

# 데이터로더

# 훈련용 데이터로더
# 훈련용이므로 셔플을 True로 설정함
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

# 검증용 데이터로더
# 검증용은 셔플이 필요하지 않음
test_loader = DataLoader(test_set,  batch_size=batch_size, shuffle=False)

## 11.6 ResNet18 불러오기

### 모델 불러오기

In [ ]:
#  라이브러리 임포트
from torchvision import models

# 사전 학습 모델 불러오기
# pretraind = True로 학습을 마친 파라미터를 동시에 불러오기
net = models.resnet18(pretrained = True)

### 모델 구조 확인

In [ ]:
# 모델 개요 표시 1

print(net)

In [ ]:
# 모델 개요 표시 2
net = net.to(device)
summary(net,(100,3,112,112))

In [ ]:
print(net.fc)
print(net.fc.in_features)

최종 레이어 함수의 변수명은 ``fc``임을 알 수 있다.

## 11.7 최종 레이어 함수 교체하기

In [ ]:
# 난수 고정
torch_seed()

# 최종 레이어 함수의 입력 차원수 확인 (e.g., ResNet-18의 경우 512)
fc_in_features = net.fc.in_features
print(f'기존 입력 차원: {fc_in_features}') # 확인용 출력

# 최종 레이어 함수를 새로운 nn.Linear로 교체
# 입력은 그대로, 출력은 우리의 클래스 개수(n_output)로 설정
net.fc = nn.Linear(fc_in_features, n_output)

# 교체 후 모델의 마지막 레이어 확인
print(net.fc)

In [ ]:
# 모델 개요 표시 1
print(net)

In [ ]:
# 모델 개요 표시 2

net = net.to(device)
summary(net,(100,3,224,224))

In [ ]:
# 손실 계산 그래프 시각화

criterion = nn.CrossEntropyLoss()
loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

In [ ]:
# 모델 개요 표시 1
print(net)

In [ ]:
# 모델 개요 표시 2
net = net.to(device)
summary(net,(100,3,112,112))

## 11.8 학습과 결과 평가

### 초기 설정

In [ ]:
# 난수 고정
torch_seed()

# 사전 학습 모델 불러오기
# pretraind = True로 학습을 마친 파라미터도 함께 불러오기
net = models.resnet18(pretrained = True)

# 최종 레이어 함수 입력 차원수 확인
fc_in_features = net.fc.in_features

# 최종 레이어 함수 교체
net.fc = nn.Linear(fc_in_features, n_output)

# GPU 사용
net = net.to(device)

# 학습률
lr = 0.001

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 최적화 함수 정의
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

# history 파일 초기화
history = np.zeros((0, 5))

### 학습

In [ ]:
# 학습
num_epochs = 5
history = fit(net, optimizer, criterion, num_epochs,
        train_loader, test_loader, device, history)

### 학습 결과 평가

In [ ]:
# 결과 요약
evaluate_history(history)

In [ ]:
# 이미지와 정답, 예측 결과를 함께 표시
show_images_labels(test_loader, classes, net, device)

## 11.9 VGG-19-BN 활용하기

### 모델 불러오기

In [ ]:
# 사전 학습 모델 불러오기
from torchvision import models
net = models.vgg19_bn(pretrained = True)

### 모델 구조 확인

In [ ]:
# 모델 개요 표시 1
print(net)

최종 레이어 함수는``classifier[6]``임을 알 수 있다.

In [ ]:
# 최종 레이어 함수 확인
print(net.classifier[6])

### 최종 레이어 함수 교체

In [ ]:
torch_seed() # 재현성을 위해 난수 다시 고정

# 최종 레이어의 입력 차원 가져오기
in_features = net.classifier[6].in_features
# 새로운 레이어로 교체
net.classifier[6] = nn.Linear(in_features, n_output)

# (참고) 책의 재현성을 위한 추가적인 모델 구조 조정
net.features = net.features[:-1]
net.avgpool = nn.Identity()

In [ ]:
# 모델 개요 표시 2
net = net.to(device)
summary(net,(100,3,112,112))

In [ ]:
# 손실 계산 그래프 시각화

criterion = nn.CrossEntropyLoss()
loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

### 초기 설정

In [ ]:
# 난수 고정
torch_seed()

# 사전 학습 모델 불러오기
net = models.vgg19_bn(pretrained = True)

# 최종 레이어 함수 교체
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, n_output)

# features 마지막의 MaxPool2d 제거
net.features = net.features[:-1]

# AdaptiveAvgPool2d 제거
net.avgpool = nn.Identity()

# 모델을 GPU로 전송
net = net.to(device)

# 학습률
lr = 0.001

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 최적화 함수 정의
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

# history 초기화
history = np.zeros((0, 5))


### 학습

In [ ]:
num_epochs = 5
history = fit(net, optimizer, criterion, num_epochs,
          train_loader, test_loader, device, history)

### 결과 확인

In [ ]:
# 결과 요약
evaluate_history(history)

In [ ]:
# 이미지와 정답, 예측 결과를 함께 표시
show_images_labels(test_loader, classes, net, device)

## 칼럼 CIFAR-10에 전이 학습을 적용한 경우

In [ ]:
# 전이 학습

# 사전 학습 모델 불러오기
net = models.resnet18(pretrained = True)

# 모든 파라미터의 경사 계산을 OFF로 설정
for param in net.parameters():
    param.requires_grad = False

# 난수 고정
torch_seed()

# 최종 레이어 함수 교체
net.fc = nn.Linear(net.fc.in_features, n_output)

# GPU 사용
net = net.to(device)

# 학습률
lr = 0.001

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 최적화 함수 정의
# 파라미터 변경은 최종 레이어 함수로 한정
optimizer = optim.SGD(net.fc.parameters(), lr=lr, momentum=0.9)

# history 파일 초기화
history = np.zeros((0, 5))

In [ ]:
# 학습
num_epochs = 5
history = fit(net, optimizer, criterion, num_epochs,
        train_loader, test_loader, device, history)

In [ ]:
# 결과 요약
evaluate_history(history)

## 칼럼 범용적인 사전 학습 모델을 작성하는 법

### 모델 불러오기

In [ ]:
# 사전 학습 모델 불러오기
from torchvision import models

net = models.vgg19_bn(pretrained = True)

### 모델 개요 표시 1

In [ ]:
print(net)

### 중간 텐서 확인

In [ ]:
# 원본 데이터 사이즈의 경우(배치사이즈 100)
net = net.to(device)
summary(net, (100, 3, 224, 224))

In [ ]:
# 실습용 데이터 사이즈의 경우(배치사이즈 100)
summary(net, (100, 3, 112, 112))

### 레이어 함수 교체하기

In [ ]:
# 난수 고정
torch_seed()

# 최종 레이어 함수 교체
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, n_output)

In [ ]:
# features의 마지막 요소(MaxPool2d)를 제거
net.features = net.features[:-1]
print(net.features)

In [ ]:
# avgpool에 위치한AdaptiveAvgPool2d을 아무것도 하지 않는 함수(nn.Identity)로 치환
net.avgpool = nn.Identity()

### 결과 확인

In [ ]:
print(net)

In [ ]:
# 실습용 데이터 사이즈로 중간 텐서 확인(배치사이즈 100)
net = net.to(device)
summary(net,(100, 3, 112, 112))